In [ ]:
%env DATA_PATH=../../../data
from api.routers.rolls import get_roll_graphs
from lib.geo import get_elevations
from lib.fit import get_sensor_data, load_fit_file
from db import *
%load_ext line_profiler

env: DATA_PATH=../../../data


In [9]:
with Session(engine) as session:
    %lprun -f get_roll_graphs -f get_elevations -f get_sensor_data get_roll_graphs(6, session)

Timer unit: 1e-09 s

Total time: 0.941593 s
File: /home/d0ugins/cmu/ec/srs/backend/src/api/routers/rolls.py
Function: get_roll_graphs at line 254

Line #      Hits         Time  Per Hit   % Time  Line Contents
   254                                           @router.get("/{roll_id}/graphs")
   255                                           def get_roll_graphs(roll_id: int, session: SessionDep):
   256         2    8348685.0 4.17e+06      0.9      roll = session.scalar(
   257         1    1506592.0 1.51e+06      0.2          select(Roll).options(selectinload(Roll.roll_files)).where(Roll.id == roll_id)
   258                                               )    
   259         1        587.0    587.0      0.0      if not roll:
   260                                                   raise HTTPException(status_code=404, detail="Roll not found")
   261                                               
   262         1      11218.0  11218.0      0.0      fit_files = [rf for rf in roll.roll_files

In [2]:
messages = load_fit_file(f"{DATA_PATH}/virbs/zr/2025-09-21-08-26-18.fit")
calibration_mesgs = messages['three_d_sensor_calibration_mesgs']
calibration_data = { m['sensor_type']: m for m in calibration_mesgs }
accel_cal = calibration_data['accelerometer']

In [6]:
%timeit get_sensor_data(accel_cal, messages['accelerometer_data_mesgs'], {'x': 'accel_x', 'y': 'accel_y', 'z': 'accel_z'})
# %timeit get_sensor_data(accel_cal, messages['accelerometer_data_mesgs'], {'x': 'accel_x', 'y': 'accel_y', 'z': 'accel_z'})

34.3 ms ± 2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
